# 3 lista SQL
### Łukasz Ozimek
### 236529

In [5]:
import pandas as pd
import numpy as np
import mysql.connector
from getpass import getpass

In [121]:
# Data and the tables created in this project
employees = {
    'emp_id':[23435,75536,78646,43235],
    'name':['John','Jane','Michael','Michelle'],
    'last_name':['Smith','Doe','Kane','Solo'],
    'age':[43,25,35,51],
    'salary':[100000,40000,60000,120000]
}

clients = {
    'client_id':[235546,667785,334432,675896,333577],
    'client_name':['Facebook','Amazon','Netflix','Google','Some company']
}

projects = {
    'client_id':[235546,667785,334432,675896,333577],
    'emp_id':[23435,75536,78646,43235,23435],
    'price':[10000,4500,80000,34000,10000]
}

emp_df = pd.DataFrame.from_dict(employees) 
client_df = pd.DataFrame.from_dict(clients)
project_df = pd.DataFrame.from_dict(projects)
display(emp_df)
display(client_df)
display(project_df)

,emp_id,name,last_name,age,salary
0,23435,John,Smith,43,100000
1,75536,Jane,Doe,25,40000
2,78646,Michael,Kane,35,60000
3,43235,Michelle,Solo,51,120000


,client_id,client_name
0,235546,Facebook
1,667785,Amazon
2,334432,Netflix
3,675896,Google
4,333577,Some company


,client_id,emp_id,price
0,235546,23435,10000
1,667785,75536,4500
2,334432,78646,80000
3,675896,43235,34000
4,333577,23435,10000


In [15]:
# Connecting to localhost with user input
mydb = mysql.connector.connect(
    host="localhost",
    user=input("Enter username: "),
    password=getpass("Enter password: "))
print(mydb)

Enter username: root
Enter password: ········


In [21]:
# Creating cursor
cursor = mydb.cursor()

def show_database_list(cursor):
    # Show database
    query = 'SHOW DATABASES'
    cursor.execute(query)
    for db in cursor:
        print(db)
    
show_database_list(cursor)

# Creating database
print('\n Database creation \n')
try:
    query = 'CREATE DATABASE excercise_database'
    cursor.execute(query)
except:
    # Deleting databse if it already exists
    query = 'DROP DATABASE excercise_database'
    cursor.execute(query)
    query = 'CREATE DATABASE excercise_database'
    cursor.execute(query)

show_database_list(cursor)

('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)

 Database creation 

('excercise_database',)
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [128]:
# Connecting to localhost database with user input
mydb = mysql.connector.connect(
    host="localhost",
    user=input("Enter username: "),
    password=getpass("Enter password: "),
    database = 'excercise_database')
print(mydb)
cursor = mydb.cursor()

Enter username: root
Enter password: ········


In [31]:
# Creating the employees table
query = """
CREATE TABLE employees(
    emp_id INT PRIMARY KEY,
    name VARCHAR(30),
    last_name VARCHAR(30),
    age INT,
    salary INT
)
"""
cursor.execute(query)
mydb.commit()

# Creating the clients table
query = """
CREATE TABLE clients(
    client_id INT PRIMARY KEY,
    client_name VARCHAR(30)
)
"""
cursor.execute(query)
mydb.commit()

# Creating the projects table
query = """
CREATE TABLE projects(
    emp_id INT,
    client_id INT,
    price INT,
    PRIMARY KEY(emp_id,client_id),
    FOREIGN KEY(emp_id) REFERENCES employees(emp_id) ON DELETE CASCADE,
    FOREIGN KEY(client_id) REFERENCES clients(client_id) ON DELETE CASCADE
)
"""
cursor.execute(query)
mydb.commit()

# Show list of tables in database
query="SHOW TABLES"
cursor.execute(query)
for tb in cursor:
    print(tb)

('clients',)
('employees',)
('projects',)


In [122]:
def df_to_query(table, dataframe):
    # Function to turn a dataframe into a SQL query (pandas to_sql() function doesn't work with mysql.connector)
    query = "INSERT INTO "+table+" ("
    string = ''
    for i in dataframe.columns:
        string = string + i+','
    string = string[0:-1]
    query = query+string+') VALUES'
    str_list = []
    for i in range(0,dataframe.shape[0]):
        str2 = ''
        df_row = dataframe[i:i+1]
        vals=df_row.values.tolist()[0]
        for val in vals:
            if type(val)==str:
                str2 = str2+"\""+val+"\","
            else:
                str2 = str2+str(val)+','
        str_list.append(str2)

    for idx,item in enumerate(str_list):
        str_list[idx] = '('+item[0:-1]+')'

    for i in str_list:
        query = query+i+','
    query = query[0:-1]
    return query

# Using the function to create the employees table
query = df_to_query('employees',emp_df)
cursor.execute(query)
mydb.commit()

In [129]:
cursor = mydb.cursor(buffered=True)
# Displaying data from the employees table
query = "SELECT * FROM employees"
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

(23435, 'John', 'Smith', 43, 100000)
(43235, 'Michelle', 'Solo', 51, 120000)
(75536, 'Jane', 'Doe', 25, 40000)
(78646, 'Michael', 'Kane', 35, 60000)


In [130]:
# Using the function to create the clients table
query = df_to_query('clients',client_df)
cursor.execute(query)
mydb.commit()


# Using the function to create the projects table
query = df_to_query('projects',project_df)
cursor.execute(query)
mydb.commit()

In [131]:
query = "SELECT * FROM clients"
cursor.execute(query)
result = cursor.fetchall()

print('\t Clients table: \n')
for x in result:
    print(x)
    
query = "SELECT * FROM projects"
cursor.execute(query)
result = cursor.fetchall()

print('\n\t Projects table: \n')
for x in result:
    print(x)

	 Clients table: 

(235546, 'Facebook')
(333577, 'Some company')
(334432, 'Netflix')
(667785, 'Amazon')
(675896, 'Google')

	 Projects table: 

(23435, 235546, 10000)
(23435, 333577, 10000)
(43235, 675896, 34000)
(75536, 667785, 4500)
(78646, 334432, 80000)


In [133]:
# Selecting all project and ordering vy price
query = "SELECT * FROM projects ORDER BY price, client_id"
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

(75536, 667785, 4500)
(23435, 235546, 10000)
(23435, 333577, 10000)
(43235, 675896, 34000)
(78646, 334432, 80000)


In [134]:
# Selecting distinct values from projects
query = "SELECT DISTINCT price FROM projects"
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

(10000,)
(34000,)
(4500,)
(80000,)


In [136]:
# Counting emloyees over 30 with salary bigger than 70000
query = "SELECT COUNT(emp_id) FROM employees WHERE salary>70000 AND age>30"
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

(2,)


In [137]:
# Getting the average project price
query = "SELECT AVG(price) FROM projects"
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

(Decimal('27700.0000'),)


In [142]:
# Selecting names and last names of employees, whose names start with J
query = "SELECT name, last_name FROM employees WHERE name LIKE 'J%'"
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

('John', 'Smith')
('Jane', 'Doe')


In [149]:
# Selecting all project and ordering vy price
query = """
    SELECT employees.name, employees.last_name, projects.client_id, projects.price 
    FROM projects JOIN employees ON employees.emp_id=projects.emp_id
"""
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

('John', 'Smith', 235546, 10000)
('John', 'Smith', 333577, 10000)
('Michelle', 'Solo', 675896, 34000)
('Jane', 'Doe', 667785, 4500)
('Michael', 'Kane', 334432, 80000)


In [158]:
# Query to select names of employees and clients related to certain projects
query = """
SELECT employees.name, employees.last_name, clients.client_name, projects.price
FROM projects
JOIN employees ON employees.emp_id=projects.emp_id
JOIN clients ON clients.client_id=projects.client_id
"""
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

('John', 'Smith', 'Facebook', 10000)
('John', 'Smith', 'Some company', 10000)
('Michelle', 'Solo', 'Google', 34000)
('Jane', 'Doe', 'Amazon', 4500)
('Michael', 'Kane', 'Netflix', 80000)


In [160]:
# Selecting client names of clients with projects over 10000
query = """
SELECT clients.client_name FROM clients WHERE client_id IN
(SELECT projects.client_id FROM projects WHERE projects.price>10000)
"""
cursor.execute(query)
result = cursor.fetchall()

for x in result:
    print(x)

('Netflix',)
('Google',)


In [195]:
# A function to allow a user to add employees
def add_employee(emp_id, name, last_name,age,salary):
    # Downloading list of employee IDs to ensure the new one is unique
    query = "SELECT emp_id FROM employees"
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Argument validation
        # ID:
    if not isinstance(emp_id, int):
        raise Exception('Employee ID must be a number')
    if emp_id<10000 or emp_id > 99999:
        raise Exception('Employee ID must have 5 digits')
    if (emp_id,) in results:
        raise Exception('Employee ID must be unique')
        
        # Name:
    if not isinstance(name, str):
        raise Exception('Employee name must be a string')
    if len(name)<2 or len(name) > 30:
        raise Exception('Employee name must be between 2-30 characters')
    name = '\"'+name+'\"'
    
        # Last name:
    if not isinstance(last_name, str):
        raise Exception('Employee name must be a string')
    if len(last_name)<2 or len(last_name) > 30:
        raise Exception('Employee name must be between 2-30 characters')
    last_name = '\"'+last_name+'\"'
    
        # Age
    if not isinstance(age, int):
        raise Exception('Employee age must be a number')
    if age<10 or age > 200:
        raise Exception('Employee must be 10-200 years old')
        
        # Salary
    if not isinstance(salary, int):
        raise Exception('Salary must be a number')
    if salary<1000:
        raise Exception('Salary must be higher than 1000')
        
    query = """
    INSERT INTO employees(emp_id, name, last_name, age, salary)
    VALUES (%s, %s, %s, %s, %s)
    """ % (emp_id, name, last_name, age, salary)
    cursor.execute(query)
    mydb.commit()


In [196]:
# Testing the employee adding function
print('Employees table before adding an employee')
query = 'SELECT * FROM employees'
cursor.execute(query)
results = cursor.fetchall()
display(pd.DataFrame(results, columns=emp_df.columns))

add_employee(23436,'Adam','Małysz',50,500000)

print('\n\nEmployees table after adding an employee')
query = 'SELECT * FROM employees'
cursor.execute(query)
results = cursor.fetchall()
result_df=pd.DataFrame(results, columns=emp_df.columns)
result_df['salary_no_tax'] = result_df['salary']*2
display(result_df)

Employees table before adding an employee


,emp_id,name,last_name,age,salary
0,23435,John,Smith,43,100000
1,43235,Michelle,Solo,51,120000
2,75536,Jane,Doe,25,40000
3,78646,Michael,Kane,35,60000




Employees table after adding an employee


,emp_id,name,last_name,age,salary,salary_no_tax
0,23435,John,Smith,43,100000,200000
1,23436,Adam,Małysz,50,500000,1000000
2,43235,Michelle,Solo,51,120000,240000
3,75536,Jane,Doe,25,40000,80000
4,78646,Michael,Kane,35,60000,120000
